In [41]:
# Importar biblioteca pyodbc:
import pyodbc

# Importar a biblioteca Pandas:
import pandas as pd
import locale
import ctypes 

# Classe para conexão com o banco de dados:
class ConexaoBD ():
    def conectar (self):
        return pyodbc.connect (driver = '{SQL Server}', server = 'DESKTOP-ATE5M9D\SQLEXPRESS', database ='empresa', trusted_connection = 'yes')
        
# Instanciar um objeto da classe para conexão com o banco de dados:
bd = ConexaoBD ()

# Pegar uma conexão com o banco de dados:
conexao = bd.conectar ()

In [71]:
# Classe Vendedor:
class Vendedor:
    def __init__ (self, conexao):
        self.conexao = conexao
   
    def moeda(self,valor):
        locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
        valor = locale.currency(valor, grouping=True, symbol=None)
        return ('R$ %s' % valor)
    
    def buscar (self):
        sql = ''' select v.cpf as CPF, v.nome as Vendedor, v.email as 'E-mail'
        from Vendedor v
        '''
        data_frame = pd.read_sql_query (sql, self.conexao)
        print (data_frame)
        print (type (data_frame))

    def buscar_Like (self):
        sql = ''' select Vendedor.cpf as CPF, Vendedor.nome as Vendedor from Vendedor
        where Vendedor.nome like '%rl%';'''
        data_frame = pd.read_sql_query (sql, self.conexao)
        print (data_frame)
        print (type (data_frame))

# Questão 01   
    def buscar_Vendedor_Por_Letra_Final_Nome (self):
        
        sql = "Select nome as Vendedor from Vendedor"
        Letra = input ("Informe uma letra para consultada do nome: ")
        sql = sql + " where nome like  '%" + Letra +  "';"
        print ('Os vendedores que possuem o nome terminado com a letra “' + Letra +  '” são: ')
        cursor = self.conexao.cursor ()
        cursor.execute (sql)
        for nome in cursor: 
            print (nome)

# Questão 02  
    def buscar_Vendedor_Por_Valor_Produto (self):
        
        sql = "Select Vendedor.nome as Vendedor,Produto.nome as Produto, VendedorProduto.preco as Preco"
        sql = sql +" From Vendedor,  VendedorProduto, Produto"
        sql = sql +" Where Vendedor.cpf = VendedorProduto.cpf_vendedor"
        sql = sql +" And VendedorProduto.id_produto = Produto.id"
        resp = ctypes.windll.user32.MessageBoxW(0, "Deseja consultar valores especificos?", "Atenção",  3)
        if(resp == 6):
            VlrMax = input ("Informe o valor máximo: ")
            VlrMin = input ("Informe o valor mínimo: ")
            sql = sql + " And (VendedorProduto.preco between "+ VlrMin +" and "+ VlrMax +")"
            print ('Os vendedores que possuem produtos com valor entre '+ VlrMin +' e '+ VlrMax +' são:')
        else:
            sql = sql + " And ((VendedorProduto.preco between 10 and 50) or (VendedorProduto.preco between 100 and 500))"
            print ('Os vendedores que possuem produtos com valor entre $10 e $50 ou entre $100 e $500 são:')

        sql = sql + " Order by VendedorProduto.preco, Vendedor.nome, Produto.nome"
        cursor = self.conexao.cursor ()
        cursor.execute (sql)
        for nome in cursor: 
            print (nome)   

# Questão 03
    def buscar_Previsao_Vendas_Por_Vendendor (self):
        resp = ctypes.windll.user32.MessageBoxW(0, "Deseja consultar um vendedor especificos?", "Atenção",  3)        
        if(resp == 6):
            NomeVendedor = input ("Informe o nome do vendendor ou 'sair' para finalizar o processo :")
        else:
            NomeVendedor = ""             

        sql = "Select Vendedor.nome as Vendedor, sum(VendedorProduto.preco * VendedorProduto.quantidade) as PrevisaoVendas"
        sql = sql +" From Vendedor,  VendedorProduto"
        sql = sql +" Where Vendedor.cpf = VendedorProduto.cpf_vendedor"        
        if (NomeVendedor != ""):  sql = sql +" And nome like  '%" + NomeVendedor +  "%'"
        sql = sql + " Group by Vendedor.nome"
        sql = sql +" Order By 2"
        
        cursor = self.conexao.cursor ()
        cursor.execute (sql)
        for PrevisaoVendas in cursor: 
            print ('O vendedor '+ PrevisaoVendas[0] +' receberá '+ self.moeda(PrevisaoVendas[1]) +' se todos os seus produtos forem vendidos.')

# Questão 04
    def buscar_Previsao_Vendas_Vendendor_Maior_Valor (self):
        
        sql = "Select top 1 Vendedor.nome as Vendedor, "
        sql = sql +"	  sum(VendedorProduto.preco * VendedorProduto.quantidade) as PrevisaoVendas "
        sql = sql +"  From Vendedor"
        sql = sql +" Inner join VendedorProduto "
        sql = sql +"    On Vendedor.cpf = VendedorProduto.cpf_vendedor "
        sql = sql +" Group by Vendedor.nome"
        sql = sql +" Order by 2 desc"

        cursor = self.conexao.cursor ()
        cursor.execute (sql)
        for PrevisaoVendas in cursor: 
            print ('O vendedor '+ PrevisaoVendas[0] +' receberá '+ self.moeda(PrevisaoVendas[1]) +' se vender todos os seus produtos sendo este o maior valor.')
           
# Questão 05
    def buscar_Previsao_Total_Vendas_Geral (self):
                
        sql = "Select sum(VendedorProduto.preco * VendedorProduto.quantidade) as PrevisaoVendas"
        sql = sql +" From Vendedor,  VendedorProduto"
        sql = sql +" Where Vendedor.cpf = VendedorProduto.cpf_vendedor"
        cursor = self.conexao.cursor ()
        cursor.execute (sql)
        for PrevisaoVendas in cursor: 
            print ('O valor total será '+ self.moeda(PrevisaoVendas[0]) +' se todos os seu produtos forem vendidos.')

# Questão 06
    def buscar_Previsao_Media_Total_Vendas_Geral (self):
        
        sql = " Select cast(avg(PrevisaoVendas) as decimal(18,2)) as MediaPrevisaoVendas"
        sql = sql +"  From (Select Vendedor.nome as Vendedor, "
        sql = sql +"               sum(VendedorProduto.preco * VendedorProduto.quantidade) as PrevisaoVendas "
        sql = sql +"          From Vendedor,  VendedorProduto Where Vendedor.cpf = VendedorProduto.cpf_vendedor Group by Vendedor.nome"
        sql = sql +"        ) as Temp"
        
        cursor = self.conexao.cursor ()
        cursor.execute (sql)
        for PrevisaoVendas in cursor: 
            print ('A média das vendas é R$ '+ self.moeda(PrevisaoVendas[0]) +' se todos os produtos forem vendidos.')

# Questão 07
    def buscar_Previsao_Vendedor_Acima_Media_Total_Vendas_Geral (self):

        sql = " Select Vendedor, PrevisaoVendas "
        sql = sql +"    From (Select Vendedor.nome as Vendedor, sum(VendedorProduto.preco * VendedorProduto.quantidade) as PrevisaoVendas "
        sql = sql +"  		  From Vendedor,  VendedorProduto "
        sql = sql +"  		 Where Vendedor.cpf = VendedorProduto.cpf_vendedor "
        sql = sql +"  		 Group by Vendedor.nome "
        sql = sql +"          ) as Temp "
        sql = sql +"   Where PrevisaoVendas > (Select cast(avg(PrevisaoVendas) as decimal(18,2)) as MediaPrevisaoVendas "
        sql = sql +"  						   From (Select Vendedor.nome as Vendedor, "
        sql = sql +"  								        sum(VendedorProduto.preco * VendedorProduto.quantidade) as PrevisaoVendas "
        sql = sql +"  								   From Vendedor,  VendedorProduto Where Vendedor.cpf = VendedorProduto.cpf_vendedor "
        sql = sql +"  								  Group by Vendedor.nome "
        sql = sql +"  								 ) as Temp) "
        
        cursor = self.conexao.cursor ()
        cursor.execute (sql)
        print ('Os vendedores com previsão total de vendas acima da média são:')    
        for PrevisaoVendas in cursor: 
            print ( PrevisaoVendas[0]  + ' com previsão de vendas de R$ '+ self.moeda(PrevisaoVendas[1]) +'.')               

In [72]:
#  1) Quais vendedores possuem o nome terminado com a letra “s”? 
#     Resp.: Elis e Iris.

# Instanciar objeto da classe ConsultasGlobais:
Vendedor = Vendedor (conexao)
# Consulta os registros do banco de dados:
Vendedor.buscar_Vendedor_Por_Letra_Final_Nome ()

Os vendedores que possuem o nome terminado com a letra “s” são: 
('Elis', )
('Iris', )


In [73]:
#  2) Quais vendedores possuem produtos com valor entre $10 e $50 ou entre $100 e $500?
#     Resp.: Heitor, Elis, Carla, Ana e Fábio.

# Instanciar objeto da classe ConsultasGlobais:

# Consulta os registros do banco de dados:
Vendedor.buscar_Vendedor_Por_Valor_Produto ()

Os vendedores que possuem produtos com valor entre $10 e $50 ou entre $100 e $500 são:
('Heitor', 'Camisa', Decimal('40'))
('Elis', 'Camisa', Decimal('45'))
('Carla', 'Camisa', Decimal('50'))
('Ana', 'Tênis', Decimal('110'))
('Fábio', 'Tênis', Decimal('120'))


In [74]:
#  3) Se Ana vender todos os seu produtos, quanto ela receberá? 
#     Resp.: $21.220.

# Consulta os registros do banco de dados:
# Vendedor.buscar()
Vendedor.buscar_Previsao_Vendas_Por_Vendendor ()

O vendedor Ana receberá R$ 21.220,00 se todos os seus produtos forem vendidos.


In [75]:
#  4) Se todos os vendedores venderem todos os seu produtos, qual será o total apurado nas vendas?
#     Resp.: $81.330.

# Consulta os registros do banco de dados:
Vendedor.buscar_Previsao_Total_Vendas_Geral()

O valor total será R$ 81.330,00 se todos os seu produtos forem vendidos.


In [66]:
#  5) Se todos os vendedores venderem todos os seu produtos, quem receberá o maior valor?
#     Resp.: Bruno, $53.000.
Vendedor.buscar_Previsao_Vendas_Vendendor_Maior_Valor ()

O vendedor Bruno receberá R$ 53.000,00 se vender todos os seus produtos sendo este o maior valor.


In [76]:
#  6) Se todos os vendedores venderem todos os seu produtos, qual será o valor médio das vendas?
#     Resp.: $11.618,57.
Vendedor.buscar_Previsao_Media_Total_Vendas_Geral()

A média das vendas é R$ R$ 11.618,57 se todos os produtos forem vendidos.


In [77]:
#  7) Se todos os vendedores venderem todos os seu produtos, quem receberá acima da média? 
#     Resp.: Bruno ($53.000) e Ana ($21.220).
Vendedor.buscar_Previsao_Vendedor_Acima_Media_Total_Vendas_Geral()

Os vendedores com previsão total de vendas acima da média são:
Ana com previsão de vendas de R$ R$ 21.220,00.
Bruno com previsão de vendas de R$ R$ 53.000,00.
